In [1]:
import torch
import os
import shutil
from pathlib import Path
from dotenv import load_dotenv
from google.cloud import storage
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [2]:
LOCAL_KEY = Path("sabari_secret.json")

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(LOCAL_KEY.resolve())
print(f"Authentication Configured: Using local key at {LOCAL_KEY.resolve()}")

Authentication Configured: Using local key at /content/sabari_secret.json


In [7]:
BASE_MODEL_ID = "defog/sqlcoder-7b-2"
GCS_ADAPTER_URI = "gs://boston311-chatbot-model/models/boston311-sqlcoder-tuned"
GCS_OUTPUT_URI = "gs://boston311-chatbot-model/models/boston311-sqlcoder-merged"

# Local temporary directories for processing
LOCAL_ADAPTER_DIR = "./temp_adapter"
LOCAL_MERGED_DIR = "./temp_merged_model"

In [8]:
def download_folder_from_gcs(gcs_uri, local_dir):
    print(f"Downloading from {gcs_uri}...")
    storage_client = storage.Client()

    parts = gcs_uri.replace("gs://", "").split("/")
    bucket_name = parts[0]
    prefix = "/".join(parts[1:])

    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)

    count = 0
    for blob in blobs:
        if blob.name.endswith("/"): continue

        rel_path = blob.name.replace(prefix, "", 1).lstrip("/")
        local_file_path = os.path.join(local_dir, rel_path)

        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        blob.download_to_filename(local_file_path)
        count += 1

    if count == 0:
        raise RuntimeError(f"No files found at {gcs_uri}. Check your bucket path permissions.")

    print(f"Downloaded {count} files to {local_dir}")

In [9]:
def upload_folder_to_gcs(local_dir, gcs_uri):
    print(f"Uploading to {gcs_uri}...")
    storage_client = storage.Client()

    parts = gcs_uri.replace("gs://", "").split("/")
    bucket_name = parts[0]
    prefix = "/".join(parts[1:])
    bucket = storage_client.bucket(bucket_name)

    count = 0
    for root, _, files in os.walk(local_dir):
        for file in files:
            local_path = os.path.join(root, file)
            rel_path = os.path.relpath(local_path, local_dir)
            blob_path = os.path.join(prefix, rel_path)

            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)
            count += 1

    print(f"Uploaded {count} files to {gcs_uri}")

In [10]:
# 1. Download Adapter from GCS
if os.path.exists(LOCAL_ADAPTER_DIR):
    shutil.rmtree(LOCAL_ADAPTER_DIR)
os.makedirs(LOCAL_ADAPTER_DIR, exist_ok=True)

try:
    download_folder_from_gcs(GCS_ADAPTER_URI, LOCAL_ADAPTER_DIR)
except Exception as e:
    print(f"Failed to download adapter: {e}")

Downloaded 7 files to ./temp_adapter


In [11]:
# 2. Load Base Model
try:
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
except Exception as e:
    print(f"Failed to load base model: {e}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [12]:
# 3. Load & Merge Adapter
try:
    model = PeftModel.from_pretrained(base_model, LOCAL_ADAPTER_DIR)
    model = model.merge_and_unload()
except Exception as e:
    print(f"Failed to merge model: {e}")

In [13]:
# 4. Save Locally First
if os.path.exists(LOCAL_MERGED_DIR):
    shutil.rmtree(LOCAL_MERGED_DIR)

model.save_pretrained(LOCAL_MERGED_DIR)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
tokenizer.save_pretrained(LOCAL_MERGED_DIR)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

('./temp_merged_model/tokenizer_config.json',
 './temp_merged_model/special_tokens_map.json',
 './temp_merged_model/tokenizer.model',
 './temp_merged_model/added_tokens.json',
 './temp_merged_model/tokenizer.json')

In [14]:
# 5. Upload to GCS
try:
    upload_folder_to_gcs(LOCAL_MERGED_DIR, GCS_OUTPUT_URI)
    print("\nMerge & Upload Complete.")
    print(f"Model ready at: {GCS_OUTPUT_URI}")
except Exception as e:
    print(f"Failed to upload model: {e}")

Uploading to gs://boston311-chatbot-model/models/boston311-sqlcoder-merged...
Uploaded 10 files to gs://boston311-chatbot-model/models/boston311-sqlcoder-merged

Merge & Upload Complete.
Model ready at: gs://boston311-chatbot-model/models/boston311-sqlcoder-merged
